# Single Frame Model

This script trains a single frame model based on an average image taken across the the forcasted month.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, LayerNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time
import numpy as np

## 1. Load Data

To begin, data for a single frame mnodel must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [2]:
training_datafile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/single_all.npy"
training_rainfallfile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/expected_all.npy"

In [3]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile)[:, 2:]

In [4]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)

In [5]:
print(training_videos.shape)

(274, 61, 121, 2)


In [6]:
training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))

#for n in range(0, 13):
#    training_rainfall[:, n] = (training_rainfall[:, n] - np.mean(training_rainfall[:, n])) / np.std(training_rainfall[:, n])
#    #training_rainfall[:, n] = (training_rainfall[:, n] - np.min(training_rainfall[:, n])) / (np.max(training_rainfall[:, n]) - np.min(training_rainfall[:, n]))

In [9]:
np.min(training_rainfall, axis=0)

array([19.34642857, 19.64878049, 24.996     ,  5.15      ,  1.045     ,
        2.2       , 11.62      ,  9.92777778,  1.76666667, 11.37142857,
       11.36521739,  5.95      ,  8.43888889])

### 1.1 DEPRECATED

This following section uses two separate datasets. Which is not required.

In [6]:
#training_datafile = "E:/31-12-2020/prepared-data/single_train.npy"
#validation_datafile = "E:/31-12-2020/prepared-data/single_valid.npy"
#
#training_rainfallfile = "E:/31-12-2020/prepared-data/expected_train.npy"
#validation_rainfallfile = "E:/31-12-2020/prepared-data/expected_valid.npy"

In [7]:
#training_videos = np.load(training_datafile)
#validation_videos = np.load(validation_datafile)
#
#training_rainfall = np.load(training_rainfallfile)[:, 2:]
#validation_rainfall = np.load(validation_rainfallfile)[:, 2:]

In [8]:
# We need them in X, Y, COLOURS
#training_videos = np.swapaxes(training_videos, 1, 2)
#training_videos = np.swapaxes(training_videos, 2, 3)
#
#validation_videos = np.swapaxes(validation_videos, 1, 2)
#validation_videos = np.swapaxes(validation_videos, 2, 3)

In [9]:
# Scale the data
#training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
#training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))
#
#validation_videos[:, :, :, 0] = (validation_videos[:, :, :, 0] - np.min(validation_videos[:, :, :, 0])) / (np.max(validation_videos[:, :, :, 0]) - np.min(validation_videos[:, :, :, 0]))
#validation_videos[:, :, :, 1] = (validation_videos[:, :, :, 1] - np.min(validation_videos[:, :, :, 1])) / (np.max(validation_videos[:, :, :, 1]) - np.min(validation_videos[:, :, :, 1]))

## 2. Model Definition

Next, a CNN model architecture is defined.

In [1]:
def model_generator(input_shape=(2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv2D(32, (2, 2), input_shape=input_shape, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second layer
    model.add(Conv2D(16, (2, 2), input_shape=input_shape, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third layer
    model.add(Conv2D(8, (2, 2), input_shape=input_shape, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(13, kernel_initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [2]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
    history = model.fit(
        xdata,
        ydata,
        batch_size=2,
        validation_split=0.3,
        callbacks=[tensorboard, es],
        epochs=100
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [3]:
# Flexible parameters
learning_rate = 0.0001

In [4]:
# Run each model multiple times
for i in range(0, 5):
    run_name = "Sing-3-{}".format(int(time.time())) # "SL_16_16_16__13__22_t-{}".format(int(time.time()))
    tb = TensorBoard(log_dir=".\\logs\\{}".format(run_name))
    model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
    history = train_model(run_name, tb, model, training_videos, training_rainfall)
    print("Final loss: {}".format(history.history["val_loss"][-1]))

NameError: name 'time' is not defined